In [1]:
import gensim
model = gensim.models.Word2Vec.load('ko/ko.bin') 
#https://github.com/Kyubyong/wordvectors word2vec 한국어 모델

In [2]:
model.wv.similarity('주식', '반찬')

0.30066097

In [3]:
model.wv.similarity('주식', '돈')

0.20880778

In [4]:
model.wv.most_similar('주식')

[('지분', 0.659886360168457),
 ('증권', 0.6218825578689575),
 ('사채', 0.617996335029602),
 ('자본금', 0.6030240058898926),
 ('현물', 0.5963727235794067),
 ('주주', 0.5927848815917969),
 ('코스닥', 0.5803253650665283),
 ('모회사', 0.579328179359436),
 ('상장', 0.5687190890312195),
 ('어음', 0.5640698075294495)]

In [5]:
import pandas as pd
import numpy as np
import os

seoul=pd.read_csv("seoul2020.csv",encoding = 'euc-kr')
services = seoul['서비스_업종_코드_명'].unique().tolist()

In [6]:
Word_vectors=model.wv
vocabs=Word_vectors.vocab.keys()
vocabs_list=list(vocabs)

In [7]:
model_services_list= []

for i in range(0,len(vocabs_list)):
    for j in range(0,len(services)):
        if vocabs_list[i] == services[j]:
            model_services_list.append(vocabs_list[i])
            break

In [8]:
len(model_services_list)

18

In [9]:
not_in_model=list(set(services) - set(model_services_list))
len(not_in_model)

45

In [11]:
service_dict = {}
service_dict['일반의원']='병원'
service_dict['당구장']='당구'
service_dict['청과상']='과일'
service_dict['일반의류']='의류'
service_dict['피부관리실']='피부'
service_dict['일식음식점']='초밥'
service_dict['부동산중개업']='부동산'
service_dict['자동차수리']='수리'
service_dict['분식전문점']='떡볶이'
service_dict['자전거 및 기타운송장비']='자전거'
service_dict['양식음식점']='스테이크'
service_dict['스포츠 강습']= '체육'
service_dict['가전제품']= '냉장고'
service_dict['스포츠클럽']='운동장'
service_dict['반찬가게']='반찬'
service_dict['조명용품']='조명'
service_dict['한의원']='한약'
service_dict['수산물판매']='수산물'
service_dict['외국어학원']='학원'
service_dict['컴퓨터및주변장치판매']='컴퓨터'
service_dict['패스트푸드점']='패스트푸드'
service_dict['커피-음료']='커피'
service_dict['치킨전문점']='치킨'
service_dict['미곡판매']='미곡'
service_dict['중식음식점']='짜장면'
service_dict['일반교습학원']='강습소'
service_dict['섬유제품']='섬유'
service_dict['운동/경기용품']='라켓'
service_dict['고시원']='방'
service_dict['치과의원']='치과'
service_dict['전자상거래업']='노트북'
service_dict['철물점']='망치'
service_dict['호프-간이주점']='술'
service_dict['가전제품수리']='검수'
service_dict['세탁소']='세탁'
service_dict['PC방']='게임'
service_dict['자동차미용']='승용차'
service_dict['한식음식점']='한식'
service_dict['육류판매']='육류'
service_dict['시계및귀금속']='귀금속'
service_dict['의료기기']='의료'
service_dict['골프연습장']='골프'
service_dict['예술학원']='예술'
service_dict['제과점']='제과'
service_dict['네일숍']='미용'

In [12]:
Word_vectors=model.wv
all_list = model_services_list + list(service_dict.values())
Word_vectors_list=[Word_vectors[v] for v in all_list]

In [15]:
len(Word_vectors_list)

63

# 클러스터링 결과로 나온 빈도수 높은 업종- 유사도 업종 찾기

In [22]:
high_services_list = ['한식음식점',
'화장품',
'의료기기',
'PC방',
'편의점',
'컴퓨터및주변장치판매'] #빈도수 높은 업종

In [23]:
in_model_high_services_list=[]

for i in range(0,len(high_services_list)):
    if high_services_list[i] in service_dict:
        in_model_high_services_list.append(service_dict[high_services_list[i]])
    else:
        in_model_high_services_list.append(high_services_list[i])
        
in_model_high_services_list

['한식', '화장품', '의료', '게임', '편의점', '컴퓨터']

In [27]:
compare_services_list =list(set(all_list) - set(in_model_high_services_list))
len(compare_services_list)
#compare_services_list

57

In [25]:
result_list = []

for i in range(0,len(in_model_high_services_list)):
    max = -1
    save_index=-1
    for j in range(0,len(compare_services_list)):
        temp = model.wv.similarity(in_model_high_services_list[i],compare_services_list[j])
        if(max<temp):
            max = temp
            save_index = j
    result_list.append(compare_services_list[save_index])
    

result_list    

['떡볶이', '의류', '의약품', '노트북', '슈퍼마켓', '노트북']

In [28]:
reverse_dict = {v:k for k,v in service_dict.items()}

real_result_list = []
for i in range(0,len(result_list)):
    if result_list[i] in reverse_dict:
        real_result_list.append(reverse_dict[result_list[i]])
    else:
         real_result_list.append(result_list[i])
real_result_list

['분식전문점', '일반의류', '의약품', '전자상거래업', '슈퍼마켓', '전자상거래업']

In [60]:
df = pd.DataFrame()
df['타겟 업종'] = high_services_list
df['유사 업종'] = real_result_list

df

,타겟 업종,유사 업종
0,한식음식점,분식전문점
1,화장품,일반의류
2,의료기기,의약품
3,PC방,전자상거래업
4,편의점,슈퍼마켓
5,컴퓨터및주변장치판매,전자상거래업


In [50]:
df['타겟 업종'][0]

'한식음식점'

In [61]:
similarity_list =[]
for i in range(0,df.shape[0]):
    num = model.wv.similarity(in_model_high_services_list[i],result_list[i])
    similarity_list.append(num)
    
df['업종간 유사도'] = similarity_list

In [62]:
df

,타겟 업종,유사 업종,업종간 유사도
0,한식음식점,분식전문점,0.610544
1,화장품,일반의류,0.769723
2,의료기기,의약품,0.517541
3,PC방,전자상거래업,0.519967
4,편의점,슈퍼마켓,0.780063
5,컴퓨터및주변장치판매,전자상거래업,0.713032
